In [1]:
import os
import tensorflow as tf
import pandas as pd
import json
import numpy as np

from pandas.io.json import json_normalize #package for flattening json in pandas df
import matplotlib.pyplot as plt


In [2]:
!ls


Notebookv2.0.ipynb pickle_files.zip   test_v2.csv        train_v2.csv


In [5]:
one_row = pd.read_csv("train_v2.csv", nrows=1)

In [6]:
one_row = list(one_row)

In [7]:
one_row

['channelGrouping',
 'customDimensions',
 'date',
 'device',
 'fullVisitorId',
 'geoNetwork',
 'hits',
 'socialEngagementType',
 'totals',
 'trafficSource',
 'visitId',
 'visitNumber',
 'visitStartTime']

In [21]:
%%time 
no_of_rows = 803178
# converter applies a converter function while you are loading the data, you are loading it as
# json.loadstring = json.loads() for each of those columns in the list
train_set = pd.read_csv("train_v2.csv", usecols=[i for i in one_row if i!= 'hits'], dtype={'fullVisitorId': 'str'}, nrows = no_of_rows)

CPU times: user 19.3 s, sys: 2.4 s, total: 21.7 s
Wall time: 27.4 s


In [22]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,1,1508198450
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,6,1508176307
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,1,1508201613
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,1,1508169851
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,1,1508190552


In [23]:
train_set.shape

(803178, 12)

In [10]:
test_set = pd.read_pickle("preprocessed_test_set_without_hits.pkl")

In [11]:
test_set.shape

(401589, 53)

In [10]:
train_set["device"][0]

'{"browser": "Firefox", "browserVersion": "not available in demo dataset", "browserSize": "not available in demo dataset", "operatingSystem": "Windows", "operatingSystemVersion": "not available in demo dataset", "isMobile": false, "mobileDeviceBranding": "not available in demo dataset", "mobileDeviceModel": "not available in demo dataset", "mobileInputSelector": "not available in demo dataset", "mobileDeviceInfo": "not available in demo dataset", "mobileDeviceMarketingName": "not available in demo dataset", "flashVersion": "not available in demo dataset", "language": "not available in demo dataset", "screenColors": "not available in demo dataset", "screenResolution": "not available in demo dataset", "deviceCategory": "desktop"}'

In [11]:
type(train_set["device"][0])

str

In [12]:
train_set.dtypes

channelGrouping         object
customDimensions        object
date                     int64
device                  object
fullVisitorId           object
geoNetwork              object
socialEngagementType    object
totals                  object
trafficSource           object
visitId                  int64
visitNumber              int64
visitStartTime           int64
dtype: object

In [13]:
type(train_set["customDimensions"][0])

str

In [14]:
train_set["customDimensions"][0]

"[{'index': '4', 'value': 'EMEA'}]"

In [24]:
train_set = train_set.join(train_set["device"].apply(json.loads).apply(pd.Series))

In [25]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,mobileDeviceBranding,mobileDeviceModel,mobileInputSelector,mobileDeviceInfo,mobileDeviceMarketingName,flashVersion,language,screenColors,screenResolution,deviceCategory
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop


In [26]:
%%time 
train_set = train_set.join(train_set["geoNetwork"].apply(json.loads).apply(pd.Series))

CPU times: user 2min 35s, sys: 727 ms, total: 2min 36s
Wall time: 2min 35s


In [27]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,subContinent,country,region,metro,city,cityId,networkDomain,latitude,longitude,networkLocation
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,...,Western Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,...,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Cupertino,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,...,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,windjammercable.net,not available in demo dataset,not available in demo dataset,not available in demo dataset
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,...,Western Asia,Turkey,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,...,Central America,Mexico,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,prod-infinitum.com.mx,not available in demo dataset,not available in demo dataset,not available in demo dataset


In [16]:
original_hits_column_train = train_set["hits"]

In [17]:
original_hits_column_train[0]

"[{'hitNumber': '1', 'time': '0', 'hour': '21', 'minute': '29', 'isInteraction': True, 'page': {'pagePath': '/home', 'hostname': 'shop.googlemerchandisestore.com', 'pageTitle': 'Home', 'pagePathLevel1': '/home', 'pagePathLevel2': '', 'pagePathLevel3': '', 'pagePathLevel4': ''}, 'appInfo': {'screenName': 'shop.googlemerchandisestore.com/home', 'landingScreenName': 'shop.googlemerchandisestore.com/home', 'exitScreenName': 'shop.googlemerchandisestore.com/home', 'screenDepth': '0'}, 'exceptionInfo': {'isFatal': True}, 'eventInfo': {'eventCategory': 'Enhanced Ecommerce', 'eventAction': 'Promotion Click'}, 'product': [], 'promotion': [{'promoId': 'Category Row 2', 'promoName': 'Accessories', 'promoCreative': 'toy.png', 'promoPosition': 'Category Row 2'}], 'promotionActionInfo': {'promoIsClick': True}, 'eCommerceAction': {'action_type': '0', 'step': '1'}, 'experiment': [], 'customVariables': [], 'customDimensions': [], 'customMetrics': [], 'type': 'EVENT', 'social': {'socialNetwork': '(not s

In [18]:
test_set.drop(["hits"], axis=1, inplace=True)

In [19]:
test_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,subContinent,country,region,metro,city,cityId,networkDomain,latitude,longitude,networkLocation
0,Organic Search,"[{'index': '4', 'value': 'APAC'}]",20180511,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7460955084541987166,"{""continent"": ""Asia"", ""subContinent"": ""Souther...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""4"", ""pageviews"": ""3"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1526099341,...,Southern Asia,India,Delhi,(not set),(not set),not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset
1,Direct,"[{'index': '4', 'value': 'North America'}]",20180511,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",460252456180441002,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""4"", ""pageviews"": ""3"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1526064483,...,Northern America,United States,California,San Francisco-Oakland-San Jose CA,San Francisco,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20180511,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3461808543879602873,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""4"", ""pageviews"": ""3"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1526067157,...,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,onlinecomputerworks.com,not available in demo dataset,not available in demo dataset,not available in demo dataset
3,Direct,"[{'index': '4', 'value': 'North America'}]",20180511,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",975129477712150630,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""4"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1526107551,...,Northern America,United States,Texas,Houston TX,Houston,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20180511,"{""browser"": ""Internet Explorer"", ""browserVersi...",8381672768065729990,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""4"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1526060254,...,Northern America,United States,California,Los Angeles CA,Irvine,not available in demo dataset,com,not available in demo dataset,not available in demo dataset,not available in demo dataset


In [28]:
train_set = train_set.join(train_set["totals"].apply(json.loads).apply(pd.Series))

In [29]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,visits,hits,pageviews,bounces,newVisits,sessionQualityDim,timeOnSite,transactions,transactionRevenue,totalTransactionRevenue
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,...,1,1,1,1,1,1,NaN,NaN,NaN,NaN
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,...,1,2,2,NaN,NaN,2,28,NaN,NaN,NaN
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,...,1,2,2,NaN,1,1,38,NaN,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,...,1,2,2,NaN,1,1,1,NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,...,1,2,2,NaN,1,1,52,NaN,NaN,NaN


In [30]:
%%time 
train_set = train_set.join(train_set["trafficSource"].apply(json.loads).apply(pd.Series))

CPU times: user 2min 35s, sys: 731 ms, total: 2min 36s
Wall time: 2min 35s


In [31]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,totalTransactionRevenue,campaign,source,medium,keyword,adwordsClickInfo,referralPath,isTrueDirect,adContent,campaignCode
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,...,NaN,(not set),google,organic,water bottle,{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,...,NaN,(not set),sites.google.com,referral,NaN,{'criteriaParameters': 'not available in demo ...,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,...,NaN,(not set),(direct),(none),NaN,{'criteriaParameters': 'not available in demo ...,NaN,True,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,...,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,...,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN


In [32]:
train_set.drop(["device", "geoNetwork", "totals", "trafficSource"], axis=1, inplace=True)

In [33]:
train_set.head()

,channelGrouping,customDimensions,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,browser,browserVersion,...,totalTransactionRevenue,campaign,source,medium,keyword,adwordsClickInfo,referralPath,isTrueDirect,adContent,campaignCode
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,...,NaN,(not set),google,organic,water bottle,{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,8934116514970143966,Not Socially Engaged,1508176307,6,1508176307,Chrome,not available in demo dataset,...,NaN,(not set),sites.google.com,referral,NaN,{'criteriaParameters': 'not available in demo ...,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,7992466427990357681,Not Socially Engaged,1508201613,1,1508201613,Chrome,not available in demo dataset,...,NaN,(not set),(direct),(none),NaN,{'criteriaParameters': 'not available in demo ...,NaN,True,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,9075655783635761930,Not Socially Engaged,1508169851,1,1508169851,Chrome,not available in demo dataset,...,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,6960673291025684308,Not Socially Engaged,1508190552,1,1508190552,Chrome,not available in demo dataset,...,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN


In [34]:
train_set.to_pickle("preprocessed_train_set_without_hits.pkl")

In [19]:
df = pd.read_pickle("preprocessed_train_set_without_hits.pkl")

In [36]:
df.head()

,channelGrouping,customDimensions,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,browser,browserVersion,...,totalTransactionRevenue,campaign,source,medium,keyword,adwordsClickInfo,referralPath,isTrueDirect,adContent,campaignCode
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,...,NaN,(not set),google,organic,water bottle,{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,8934116514970143966,Not Socially Engaged,1508176307,6,1508176307,Chrome,not available in demo dataset,...,NaN,(not set),sites.google.com,referral,NaN,{'criteriaParameters': 'not available in demo ...,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,7992466427990357681,Not Socially Engaged,1508201613,1,1508201613,Chrome,not available in demo dataset,...,NaN,(not set),(direct),(none),NaN,{'criteriaParameters': 'not available in demo ...,NaN,True,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,9075655783635761930,Not Socially Engaged,1508169851,1,1508169851,Chrome,not available in demo dataset,...,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,6960673291025684308,Not Socially Engaged,1508190552,1,1508190552,Chrome,not available in demo dataset,...,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN


In [42]:
one_row

['channelGrouping',
 'customDimensions',
 'date',
 'device',
 'fullVisitorId',
 'geoNetwork',
 'hits',
 'socialEngagementType',
 'totals',
 'trafficSource',
 'visitId',
 'visitNumber',
 'visitStartTime']

In [57]:
first_row = pd.read_csv("train_v2.csv", nrows=1, skiprows=no_of_rows, header=None, names=one_row)

In [58]:
first_row

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7779444718212527610,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250518,2,1507250518


In [6]:
%%time 
no_of_rows = 803178
# converter applies a converter function while you are loading the data, you are loading it as
# json.loadstring = json.loads() for each of those columns in the list
train_set = pd.read_csv("train_v2.csv",header=None, names=one_row, usecols=[i for i in one_row if i!= 'hits'], dtype={'fullVisitorId': 'str'}, skiprows= no_of_rows)

CPU times: user 33.1 s, sys: 4.48 s, total: 37.6 s
Wall time: 53.4 s


In [7]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7779444718212527610,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250518,2,1507250518
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",8127326206655250634,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250122,1,1507250122
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8512418246113365791,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507237813,1,1507237813
3,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",4331867527339252882,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250491,1,1507250491
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",6311336399415104146,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""8"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507243409,1,1507243409


In [8]:
train_set.shape

(905160, 12)

In [9]:
%%time
train_set = train_set.join(train_set["device"].apply(json.loads).apply(pd.Series))

CPU times: user 2min 43s, sys: 2.53 s, total: 2min 46s
Wall time: 2min 46s


In [10]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,mobileDeviceBranding,mobileDeviceModel,mobileInputSelector,mobileDeviceInfo,mobileDeviceMarketingName,flashVersion,language,screenColors,screenResolution,deviceCategory
0,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7779444718212527610,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250518,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",8127326206655250634,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250122,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,tablet
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8512418246113365791,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507237813,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
3,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",4331867527339252882,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250491,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",6311336399415104146,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""8"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507243409,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile


In [11]:
%%time 
train_set = train_set.join(train_set["geoNetwork"].apply(json.loads).apply(pd.Series))

CPU times: user 2min 55s, sys: 778 ms, total: 2min 56s
Wall time: 2min 55s


In [12]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,subContinent,country,region,metro,city,cityId,networkDomain,latitude,longitude,networkLocation
0,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7779444718212527610,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250518,...,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,spcsdns.net,not available in demo dataset,not available in demo dataset,not available in demo dataset
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",8127326206655250634,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250122,...,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8512418246113365791,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507237813,...,Northern America,Canada,British Columbia,(not set),Vancouver,not available in demo dataset,bigpipeinc.com,not available in demo dataset,not available in demo dataset,not available in demo dataset
3,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",4331867527339252882,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250491,...,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",6311336399415104146,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""8"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507243409,...,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Sunnyvale,not available in demo dataset,juniper.net,not available in demo dataset,not available in demo dataset,not available in demo dataset


In [13]:
%%time
train_set = train_set.join(train_set["totals"].apply(json.loads).apply(pd.Series))

CPU times: user 2min 44s, sys: 751 ms, total: 2min 45s
Wall time: 2min 44s


In [14]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,visits,hits,pageviews,timeOnSite,sessionQualityDim,newVisits,transactions,transactionRevenue,totalTransactionRevenue,bounces
0,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7779444718212527610,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250518,...,1,5,5,170,1,NaN,NaN,NaN,NaN,NaN
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",8127326206655250634,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250122,...,1,6,6,190,1,1,NaN,NaN,NaN,NaN
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8512418246113365791,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507237813,...,1,6,6,20,1,1,NaN,NaN,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",4331867527339252882,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250491,...,1,6,6,161,1,1,NaN,NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",6311336399415104146,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""8"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507243409,...,1,8,8,270,2,1,NaN,NaN,NaN,NaN


In [15]:
%%time 
train_set = train_set.join(train_set["trafficSource"].apply(json.loads).apply(pd.Series))

CPU times: user 2min 50s, sys: 832 ms, total: 2min 50s
Wall time: 2min 50s


In [16]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,totalTransactionRevenue,bounces,campaign,source,medium,keyword,adwordsClickInfo,isTrueDirect,referralPath,adContent
0,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7779444718212527610,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250518,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,True,NaN,NaN
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",8127326206655250634,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250122,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8512418246113365791,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507237813,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",4331867527339252882,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250491,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",6311336399415104146,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""8"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507243409,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN


In [17]:
train_set.drop(["device", "geoNetwork", "totals", "trafficSource"], axis=1, inplace=True)

In [18]:
train_set.head()

,channelGrouping,customDimensions,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,browser,browserVersion,...,totalTransactionRevenue,bounces,campaign,source,medium,keyword,adwordsClickInfo,isTrueDirect,referralPath,adContent
0,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,7779444718212527610,Not Socially Engaged,1507250518,2,1507250518,Chrome,not available in demo dataset,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,True,NaN,NaN
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,8127326206655250634,Not Socially Engaged,1507250122,1,1507250122,Safari,not available in demo dataset,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,8512418246113365791,Not Socially Engaged,1507237813,1,1507237813,Chrome,not available in demo dataset,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,4331867527339252882,Not Socially Engaged,1507250491,1,1507250491,Chrome,not available in demo dataset,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,6311336399415104146,Not Socially Engaged,1507243409,1,1507243409,Safari,not available in demo dataset,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN


In [20]:
train_set.to_pickle("preprocessed_train_set_without_hits2.pkl")

In [21]:
df2 = pd.read_pickle("preprocessed_train_set_without_hits2.pkl")

In [22]:
df.keys()

Index(['channelGrouping', 'customDimensions', 'date', 'fullVisitorId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'browser', 'browserVersion', 'browserSize', 'operatingSystem',
       'operatingSystemVersion', 'isMobile', 'mobileDeviceBranding',
       'mobileDeviceModel', 'mobileInputSelector', 'mobileDeviceInfo',
       'mobileDeviceMarketingName', 'flashVersion', 'language', 'screenColors',
       'screenResolution', 'deviceCategory', 'continent', 'subContinent',
       'country', 'region', 'metro', 'city', 'cityId', 'networkDomain',
       'latitude', 'longitude', 'networkLocation', 'visits', 'hits',
       'pageviews', 'bounces', 'newVisits', 'sessionQualityDim', 'timeOnSite',
       'transactions', 'transactionRevenue', 'totalTransactionRevenue',
       'campaign', 'source', 'medium', 'keyword', 'adwordsClickInfo',
       'referralPath', 'isTrueDirect', 'adContent', 'campaignCode'],
      dtype='object')

In [23]:
df2.keys()

Index(['channelGrouping', 'customDimensions', 'date', 'fullVisitorId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'browser', 'browserVersion', 'browserSize', 'operatingSystem',
       'operatingSystemVersion', 'isMobile', 'mobileDeviceBranding',
       'mobileDeviceModel', 'mobileInputSelector', 'mobileDeviceInfo',
       'mobileDeviceMarketingName', 'flashVersion', 'language', 'screenColors',
       'screenResolution', 'deviceCategory', 'continent', 'subContinent',
       'country', 'region', 'metro', 'city', 'cityId', 'networkDomain',
       'latitude', 'longitude', 'networkLocation', 'visits', 'hits',
       'pageviews', 'timeOnSite', 'sessionQualityDim', 'newVisits',
       'transactions', 'transactionRevenue', 'totalTransactionRevenue',
       'bounces', 'campaign', 'source', 'medium', 'keyword',
       'adwordsClickInfo', 'isTrueDirect', 'referralPath', 'adContent'],
      dtype='object')

In [25]:
df.keys() == df2.keys()

ValueError: Lengths must match to compare

In [28]:
df2 = list(df2.keys())

In [32]:
df2

['channelGrouping',
 'customDimensions',
 'date',
 'fullVisitorId',
 'socialEngagementType',
 'visitId',
 'visitNumber',
 'visitStartTime',
 'browser',
 'browserVersion',
 'browserSize',
 'operatingSystem',
 'operatingSystemVersion',
 'isMobile',
 'mobileDeviceBranding',
 'mobileDeviceModel',
 'mobileInputSelector',
 'mobileDeviceInfo',
 'mobileDeviceMarketingName',
 'flashVersion',
 'language',
 'screenColors',
 'screenResolution',
 'deviceCategory',
 'continent',
 'subContinent',
 'country',
 'region',
 'metro',
 'city',
 'cityId',
 'networkDomain',
 'latitude',
 'longitude',
 'networkLocation',
 'visits',
 'hits',
 'pageviews',
 'timeOnSite',
 'sessionQualityDim',
 'newVisits',
 'transactions',
 'transactionRevenue',
 'totalTransactionRevenue',
 'bounces',
 'campaign',
 'source',
 'medium',
 'keyword',
 'adwordsClickInfo',
 'isTrueDirect',
 'referralPath',
 'adContent']

In [34]:
df = list(df.keys())

In [33]:
list(df.keys()) == df2[:-1]

False

In [35]:
for one, two in zip(df, df2):
    if one != two:
        print(one, two)

bounces timeOnSite
newVisits sessionQualityDim
sessionQualityDim newVisits
timeOnSite transactions
transactions transactionRevenue
transactionRevenue totalTransactionRevenue
totalTransactionRevenue bounces
referralPath isTrueDirect
isTrueDirect referralPath


In [44]:
df = sorted(df)

In [45]:
df2 = sorted(df2)

In [46]:
for one, two in zip(df, df2):
    if one != two:
        print(one, two)

campaignCode channelGrouping
channelGrouping city
city cityId
cityId continent
continent country
country customDimensions
customDimensions date
date deviceCategory
deviceCategory flashVersion
flashVersion fullVisitorId
fullVisitorId hits
hits isMobile
isMobile isTrueDirect
isTrueDirect keyword
keyword language
language latitude
latitude longitude
longitude medium
medium metro
metro mobileDeviceBranding
mobileDeviceBranding mobileDeviceInfo
mobileDeviceInfo mobileDeviceMarketingName
mobileDeviceMarketingName mobileDeviceModel
mobileDeviceModel mobileInputSelector
mobileInputSelector networkDomain
networkDomain networkLocation
networkLocation newVisits
newVisits operatingSystem
operatingSystem operatingSystemVersion
operatingSystemVersion pageviews
pageviews referralPath
referralPath region
region screenColors
screenColors screenResolution
screenResolution sessionQualityDim
sessionQualityDim socialEngagementType
socialEngagementType source
source subContinent
subContinent timeOnSite
time

In [49]:
df2

['adContent',
 'adwordsClickInfo',
 'bounces',
 'browser',
 'browserSize',
 'browserVersion',
 'campaign',
 'channelGrouping',
 'city',
 'cityId',
 'continent',
 'country',
 'customDimensions',
 'date',
 'deviceCategory',
 'flashVersion',
 'fullVisitorId',
 'hits',
 'isMobile',
 'isTrueDirect',
 'keyword',
 'language',
 'latitude',
 'longitude',
 'medium',
 'metro',
 'mobileDeviceBranding',
 'mobileDeviceInfo',
 'mobileDeviceMarketingName',
 'mobileDeviceModel',
 'mobileInputSelector',
 'networkDomain',
 'networkLocation',
 'newVisits',
 'operatingSystem',
 'operatingSystemVersion',
 'pageviews',
 'referralPath',
 'region',
 'screenColors',
 'screenResolution',
 'sessionQualityDim',
 'socialEngagementType',
 'source',
 'subContinent',
 'timeOnSite',
 'totalTransactionRevenue',
 'transactionRevenue',
 'transactions',
 'visitId',
 'visitNumber',
 'visitStartTime',
 'visits']

In [48]:
df

['adContent',
 'adwordsClickInfo',
 'bounces',
 'browser',
 'browserSize',
 'browserVersion',
 'campaign',
 'campaignCode',
 'channelGrouping',
 'city',
 'cityId',
 'continent',
 'country',
 'customDimensions',
 'date',
 'deviceCategory',
 'flashVersion',
 'fullVisitorId',
 'hits',
 'isMobile',
 'isTrueDirect',
 'keyword',
 'language',
 'latitude',
 'longitude',
 'medium',
 'metro',
 'mobileDeviceBranding',
 'mobileDeviceInfo',
 'mobileDeviceMarketingName',
 'mobileDeviceModel',
 'mobileInputSelector',
 'networkDomain',
 'networkLocation',
 'newVisits',
 'operatingSystem',
 'operatingSystemVersion',
 'pageviews',
 'referralPath',
 'region',
 'screenColors',
 'screenResolution',
 'sessionQualityDim',
 'socialEngagementType',
 'source',
 'subContinent',
 'timeOnSite',
 'totalTransactionRevenue',
 'transactionRevenue',
 'transactions',
 'visitId',
 'visitNumber',
 'visitStartTime',
 'visits']

In [40]:
d = {}
for one in df:
    d[one] = 1

In [41]:
d2 = {}
for two in df2:
    d2[two] = 1

In [42]:
d.keys()

dict_keys(['channelGrouping', 'customDimensions', 'date', 'fullVisitorId', 'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime', 'browser', 'browserVersion', 'browserSize', 'operatingSystem', 'operatingSystemVersion', 'isMobile', 'mobileDeviceBranding', 'mobileDeviceModel', 'mobileInputSelector', 'mobileDeviceInfo', 'mobileDeviceMarketingName', 'flashVersion', 'language', 'screenColors', 'screenResolution', 'deviceCategory', 'continent', 'subContinent', 'country', 'region', 'metro', 'city', 'cityId', 'networkDomain', 'latitude', 'longitude', 'networkLocation', 'visits', 'hits', 'pageviews', 'bounces', 'newVisits', 'sessionQualityDim', 'timeOnSite', 'transactions', 'transactionRevenue', 'totalTransactionRevenue', 'campaign', 'source', 'medium', 'keyword', 'adwordsClickInfo', 'referralPath', 'isTrueDirect', 'adContent', 'campaignCode'])

In [43]:
d2.keys()

dict_keys(['channelGrouping', 'customDimensions', 'date', 'fullVisitorId', 'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime', 'browser', 'browserVersion', 'browserSize', 'operatingSystem', 'operatingSystemVersion', 'isMobile', 'mobileDeviceBranding', 'mobileDeviceModel', 'mobileInputSelector', 'mobileDeviceInfo', 'mobileDeviceMarketingName', 'flashVersion', 'language', 'screenColors', 'screenResolution', 'deviceCategory', 'continent', 'subContinent', 'country', 'region', 'metro', 'city', 'cityId', 'networkDomain', 'latitude', 'longitude', 'networkLocation', 'visits', 'hits', 'pageviews', 'timeOnSite', 'sessionQualityDim', 'newVisits', 'transactions', 'transactionRevenue', 'totalTransactionRevenue', 'bounces', 'campaign', 'source', 'medium', 'keyword', 'adwordsClickInfo', 'isTrueDirect', 'referralPath', 'adContent'])

In [2]:
df= pd.read_pickle("preprocessed_train_set_without_hits.pkl")

In [54]:
df.drop(["campaignCode"], axis=1, inplace=True)

In [55]:
df.to_pickle("preprocessed_train_set_without_hits.pkl")

In [60]:
tf = pd.read_pickle("preprocessed_test_set_without_hits.pkl")

In [61]:
tf = sorted(list(tf.keys()))

In [ ]:
original_hits_column.to_pickle("test_set_hits_column.pkl")

In [3]:
df.shape

(803178, 53)

In [22]:
for column in df.keys():
    print(df.groupby(column).size())
    print("\n")
    print("\n")
    print(type(df.groupby(column).size()))
    a = df.groupby(column).size()
    for k, v in a.iteritems():
        print(k, v)
    break
    

channelGrouping
(Other)               52
Affiliates         14892
Direct            130066
Display            23662
Organic Search    338636
Paid Search        21531
Referral          101174
Social            173165
dtype: int64




<class 'pandas.core.series.Series'>
(Other) 52
Affiliates 14892
Direct 130066
Display 23662
Organic Search 338636
Paid Search 21531
Referral 101174
Social 173165


In [4]:
df2 = pd.read_pickle("preprocessed_train_set_without_hits2.pkl")

In [15]:
for column in df2.keys():
    print(df2.groupby(column).size())
    print("\n")
    print("\n")
    break

channelGrouping
(Other)               85
Affiliates         18023
Direct            143068
Display            27621
Organic Search    400328
Paid Search        24096
Referral          110133
Social            181806
dtype: int64






In [5]:
total = []
xaxis = []
column_names = list(df.keys())

In [7]:
column_names

['channelGrouping',
 'customDimensions',
 'date',
 'fullVisitorId',
 'socialEngagementType',
 'visitId',
 'visitNumber',
 'visitStartTime',
 'browser',
 'browserVersion',
 'browserSize',
 'operatingSystem',
 'operatingSystemVersion',
 'isMobile',
 'mobileDeviceBranding',
 'mobileDeviceModel',
 'mobileInputSelector',
 'mobileDeviceInfo',
 'mobileDeviceMarketingName',
 'flashVersion',
 'language',
 'screenColors',
 'screenResolution',
 'deviceCategory',
 'continent',
 'subContinent',
 'country',
 'region',
 'metro',
 'city',
 'cityId',
 'networkDomain',
 'latitude',
 'longitude',
 'networkLocation',
 'visits',
 'hits',
 'pageviews',
 'bounces',
 'newVisits',
 'sessionQualityDim',
 'timeOnSite',
 'transactions',
 'transactionRevenue',
 'totalTransactionRevenue',
 'campaign',
 'source',
 'medium',
 'keyword',
 'adwordsClickInfo',
 'referralPath',
 'isTrueDirect',
 'adContent']

In [14]:
df.groupby("socialEngagementType").size()

socialEngagementType
Not Socially Engaged    803178
dtype: int64

In [13]:
df2.groupby("socialEngagementType").size()

socialEngagementType
Not Socially Engaged    905160
dtype: int64

In [15]:
df.groupby("visitNumber").size()

visitNumber
1      616774
2       84785
3       32971
4       17705
5       10766
6        7277
7        5192
8        3894
9        2979
10       2387
11       1884
12       1559
13       1308
14       1058
15        914
16        829
17        672
18        617
19        581
20        490
21        446
22        388
23        362
24        321
25        308
26        275
27        267
28        234
29        193
30        187
        ...  
403         1
404         1
407         1
408         1
413         1
414         1
416         1
418         1
420         1
421         1
422         1
423         1
424         1
425         1
428         1
429         1
432         1
440         1
441         1
443         1
445         1
447         1
448         1
449         1
450         1
452         1
453         1
455         1
456         1
457         1
Length: 389, dtype: int64

In [16]:
df2.groupby("visitNumber").size()

visitNumber
1      690656
2       97758
3       37991
4       20181
5       12548
6        8425
7        6002
8        4444
9        3371
10       2631
11       2123
12       1724
13       1421
14       1253
15       1067
16        908
17        840
18        729
19        614
20        583
21        509
22        478
23        404
24        395
25        353
26        328
27        282
28        272
29        273
30        249
        ...  
393         1
394         1
397         2
398         1
399         1
405         1
406         1
409         1
410         1
411         1
412         1
415         1
417         1
419         1
426         1
427         1
430         1
431         1
433         1
434         1
435         1
436         1
437         1
438         1
439         1
442         1
444         1
446         1
451         1
454         1
Length: 401, dtype: int64

In [17]:
df.groupby("browser").size()

browser
(not set)                                                    2
+Simple Browser                                              4
;__CT_JOB_ID__:2547db0b-ec43-452a-a0d4-ff42b7dc7907;         1
;__CT_JOB_ID__:4333777f-bb0c-4a18-935e-df5658dbce2d;         1
;__CT_JOB_ID__:58e2ecba-7666-4a10-b498-8216457ce472;         1
;__CT_JOB_ID__:65da7e5f-0f05-4b5d-8d31-1f4d470a2b82;         1
;__CT_JOB_ID__:6e9dcf2f-f58f-4938-91e3-77e00868177b;         1
;__CT_JOB_ID__:7e575295-571e-4e82-9254-7f2c8bbb9183;         1
;__CT_JOB_ID__:89e59554-ad41-4e94-957b-f12bd012530c;         1
;__CT_JOB_ID__:a4f837b8-8d78-4c42-ba9a-d870cf1a4a7e;         1
;__CT_JOB_ID__:a80e8e16-6e98-455b-885a-a4dd40f3d344;         1
;__CT_JOB_ID__:d14534ff-e2fc-4692-92aa-e34508f1c418;         1
;__CT_JOB_ID__:dd6177aa-1baa-4007-9b38-b7cab4f7611c;         1
ADM                                                          1
Amazon Silk                                                708
Amazon.com                                     

In [18]:
df2.groupby("browser").size()


browser
(not set)                                                   10
+Simple Browser                                              4
0                                                            7
;__CT_JOB_ID__:0a075729-93a5-43d0-9638-4cbd41d5f5a5;         1
;__CT_JOB_ID__:0b39e7ca-1431-42e3-ba1f-9d8951a65840;         1
;__CT_JOB_ID__:2e0eca60-83ab-482d-bb81-343d113254fb;         1
;__CT_JOB_ID__:76fd1acb-e365-43c0-b967-908bcf5d5b59;         1
;__CT_JOB_ID__:85da5736-a78e-45a9-837e-f5a53e5cd725;         1
;__CT_JOB_ID__:97909e28-4228-4b55-8ad5-cc791f2b583c;         1
;__CT_JOB_ID__:a24a8978-e5e8-4dc9-af66-c4ed89ea25d7;         1
;__CT_JOB_ID__:a7ed0808-e70c-4b19-b1a3-1018bbb7dc7f;         1
;__CT_JOB_ID__:fe02e46f-b6ae-41f1-8563-3b40bbb623a9;         1
Amazon Silk                                                759
Amazon.com                                                   1
Android Browser                                            476
Android Runtime                                

In [19]:
df.groupby("browserVersion").size()

browserVersion
not available in demo dataset    803178
dtype: int64

In [20]:
df2.groupby("browserVersion").size()


browserVersion
not available in demo dataset    905160
dtype: int64

In [21]:
df.groupby("browserSize").size()

browserSize
not available in demo dataset    803178
dtype: int64

In [22]:
df2.groupby("browserSize").size()

browserSize
not available in demo dataset    905160
dtype: int64

In [23]:
df.groupby("operatingSystem").size()

operatingSystem
(not set)             5306
Android             138739
BlackBerry             169
Chrome OS            23748
Firefox OS              52
FreeBSD                  8
Linux                30282
Macintosh           210840
NTT DoCoMo               1
Nintendo 3DS             5
Nintendo Wii            53
Nintendo WiiU           19
Nokia                    1
OS/2                    98
OpenBSD                  1
Playstation Vita         3
Samsung                420
SunOS                    2
SymbianOS                1
Tizen                  310
Windows             292320
Windows Phone          785
Xbox                    58
iOS                  99957
dtype: int64

In [24]:
df2.groupby("operatingSystem").size()

operatingSystem
(not set)             6509
Android             160648
BlackBerry             181
Chrome OS            27570
Firefox OS              45
FreeBSD                 10
Linux                33689
Macintosh           227674
Nintendo 3DS             4
Nintendo Wii            47
Nintendo WiiU           32
Nokia                    2
OS/2                   101
OpenBSD                  2
Playstation Vita        12
Samsung                491
SunOS                    1
SymbianOS                1
Tizen                  399
Windows             327400
Windows Phone          890
Xbox                    75
iOS                 119377
dtype: int64

In [25]:
df.groupby("operatingSystemVersion").size()

operatingSystemVersion
not available in demo dataset    803178
dtype: int64

In [26]:
df2.groupby("operatingSystemVersion").size()

operatingSystemVersion
not available in demo dataset    905160
dtype: int64

In [27]:
df.groupby("isMobile").size()

isMobile
False    556257
True     246921
dtype: int64

In [28]:
df2.groupby("isMobile").size()

isMobile
False    615413
True     289747
dtype: int64

In [29]:
df.groupby("mobileDeviceBranding").size()

mobileDeviceBranding
not available in demo dataset    803178
dtype: int64

In [30]:
df2.groupby("mobileDeviceBranding").size()

mobileDeviceBranding
not available in demo dataset    905160
dtype: int64

In [31]:
df.groupby("mobileDeviceModel").size()

mobileDeviceModel
not available in demo dataset    803178
dtype: int64

In [32]:
df2.groupby("mobileDeviceModel").size()

mobileDeviceModel
not available in demo dataset    905160
dtype: int64

In [33]:
df.groupby("mobileInputSelector").size()

mobileInputSelector
not available in demo dataset    803178
dtype: int64

In [34]:
df2.groupby("mobileInputSelector").size()

mobileInputSelector
not available in demo dataset    905160
dtype: int64

In [35]:
df.groupby("mobileDeviceInfo").size()

mobileDeviceInfo
not available in demo dataset    803178
dtype: int64

In [36]:
df2.groupby("mobileDeviceInfo").size()

mobileDeviceInfo
not available in demo dataset    905160
dtype: int64

In [37]:
df.groupby("mobileDeviceMarketingName").size()

mobileDeviceMarketingName
not available in demo dataset    803178
dtype: int64

In [38]:
df2.groupby("mobileDeviceMarketingName").size()

mobileDeviceMarketingName
not available in demo dataset    905160
dtype: int64

In [39]:
df.groupby("flashVersion").size()

flashVersion
not available in demo dataset    803178
dtype: int64

In [40]:
df2.groupby("flashVersion").size()

flashVersion
not available in demo dataset    905160
dtype: int64

In [41]:
df.groupby("language").size()

language
not available in demo dataset    803178
dtype: int64

In [42]:
df2.groupby("language").size()

language
not available in demo dataset    905160
dtype: int64

In [43]:
df.groupby("screenColors").size()

screenColors
not available in demo dataset    803178
dtype: int64

In [44]:
df2.groupby("screenColors").size()

screenColors
not available in demo dataset    905160
dtype: int64

In [45]:
df.groupby("screenResolution").size()

screenResolution
not available in demo dataset    803178
dtype: int64

In [46]:
df2.groupby("screenResolution").size()

screenResolution
not available in demo dataset    905160
dtype: int64

In [47]:
df.groupby("deviceCategory").size()

deviceCategory
desktop    556219
mobile     216414
tablet      30545
dtype: int64

In [48]:
df2.groupby("deviceCategory").size()

deviceCategory
desktop    615360
mobile     254923
tablet      34877
dtype: int64

In [49]:
df.groupby("continent").size()

continent
(not set)      1135
Africa        16070
Americas     413373
Asia         189611
Europe       170295
Oceania       12694
dtype: int64

In [50]:
df2.groupby("continent").size()

continent
(not set)      1382
Africa        19411
Americas     464031
Asia         207108
Europe       197742
Oceania       15486
dtype: int64

In [51]:
df.groupby("subContinent").size()

subContinent
(not set)               1135
Australasia            12563
Caribbean               2131
Central America        14477
Central Asia             965
Eastern Africa          1905
Eastern Asia           41953
Eastern Europe         35020
Melanesia                 65
Micronesian Region        51
Middle Africa            462
Northern Africa         7798
Northern America      359676
Northern Europe        50975
Polynesia                 15
South America          37089
Southeast Asia         61583
Southern Africa         2621
Southern Asia          55655
Southern Europe        31122
Western Africa          3284
Western Asia           29455
Western Europe         53178
dtype: int64

In [52]:
df2.groupby("subContinent").size()

subContinent
(not set)               1382
Australasia            15304
Caribbean               2251
Central America        15087
Central Asia            1020
Eastern Africa          2289
Eastern Asia           49119
Eastern Europe         38987
Melanesia                 68
Micronesian Region        77
Middle Africa            465
Northern Africa         9457
Northern America      408670
Northern Europe        60718
Polynesia                 37
South America          38023
Southeast Asia         60051
Southern Africa         3398
Southern Asia          65407
Southern Europe        36062
Western Africa          3802
Western Asia           31511
Western Europe         61975
dtype: int64

In [53]:
df.groupby("country").size()

country
(not set)                   1135
Afghanistan                   57
Albania                      388
Algeria                     1555
American Samoa                 1
Andorra                       20
Angola                        56
Anguilla                       1
Antigua & Barbuda              9
Argentina                   4757
Armenia                      217
Aruba                         10
Australia                  10638
Austria                     2311
Azerbaijan                   375
Bahamas                       45
Bahrain                      204
Bangladesh                  2234
Barbados                      67
Belarus                      806
Belgium                     4047
Belize                        24
Benin                         67
Bermuda                       30
Bhutan                        12
Bolivia                      310
Bosnia & Herzegovina         692
Botswana                      19
Brazil                     18076
British Virgin Islands         3
  

In [54]:
df2.groupby("country").size()

country
(not set)                   1382
Afghanistan                   76
Albania                      380
Algeria                     1649
American Samoa                 1
Andorra                       25
Angola                        62
Antigua & Barbuda              5
Argentina                   5371
Armenia                      225
Aruba                         32
Australia                  13022
Austria                     2839
Azerbaijan                   425
Bahamas                       56
Bahrain                      193
Bangladesh                  2558
Barbados                      62
Belarus                      875
Belgium                     4617
Belize                        26
Benin                         68
Bermuda                       26
Bhutan                        14
Bolivia                      334
Bosnia & Herzegovina         772
Botswana                      21
Brazil                     17356
British Virgin Islands         4
Brunei                        61
  

In [55]:
df.groupby("region").size()

region
(not set)                         22878
Abu Dhabi                            57
Adana                               113
Aguascalientes                        6
Aichi Prefecture                     33
Akershus                             12
Al Madinah Province                   9
Alabama                              51
Alberta                             528
Alexandria Governorate              156
Algiers Province                     25
Almaty Province                      21
Amman Governorate                   116
Andalusia                           182
Andhra Pradesh                       18
Ankara                             1046
Antalya                             151
Antioquia                           184
Aquitaine                            13
Aragon                              103
Arequipa                             16
Arizona                             587
Assam                                84
Asturias                             12
Atlantico                        

In [56]:
df2.groupby("region").size()

region
(not set)                         26896
Abruzzo                               6
Abu Dhabi                            48
Adana                               152
Aichi Prefecture                     21
Alabama                              33
Alaska                               12
Alba County                          21
Alberta                             551
Alexandria Governorate              337
Algiers Province                     21
Almaty Province                      12
Amman Governorate                   111
Andalusia                           185
Andhra Pradesh                        6
Ankara                              957
Antalya                              86
Antioquia                           109
Apulia                                6
Aquitaine                            55
Aragon                               43
Arequipa                             14
Arizona                             947
Assam                                75
Asturias                         

In [57]:
df.groupby("metro").size()

metro
(not set)                                 184841
Albany-Schenectady-Troy NY                    59
Atlanta GA                                  2591
Austin TX                                   3565
Baltimore MD                                  47
Birmingham (Ann and Tusc) AL                  15
Boise ID                                      23
Boston MA-Manchester NH                     2767
Buffalo NY                                     5
Butte-Bozeman MT                              48
Central Scotland                              45
Charleston SC                                 60
Charlotte NC                                1010
Chattanooga TN                                44
Chicago IL                                  7235
Chico-Redding CA                              46
Cincinnati OH                                 92
Cleveland-Akron (Canton) OH                  145
Columbus GA                                   29
Columbus OH                                  440
Dallas-Ft. Wor

In [58]:
df2.groupby("metro").size()

metro
(not set)                             202055
Abilene-Sweetwater TX                     10
Albany-Schenectady-Troy NY                89
Anchorage AK                              12
Atlanta GA                              2873
Augusta GA                                 7
Austin TX                               3724
Baltimore MD                              49
Birmingham (Ann and Tusc) AL              15
Boise ID                                  47
Boston MA-Manchester NH                 3101
Butte-Bozeman MT                          78
Central Scotland                          67
Champaign & Springfield-Decatur IL         6
Charleston SC                             58
Charlotte NC                             901
Charlottesville VA                        13
Chicago IL                              8065
Chico-Redding CA                          77
Cincinnati OH                            139
Cleveland-Akron (Canton) OH              127
Colorado Springs-Pueblo CO                16
Colu

In [59]:
df.groupby("city").size()

city
's Hertogenbosch                      7
(not set)                         31079
Aachen                                9
Aalborg                              39
Aberdeen                             21
Abu Dhabi                            57
Accra                                 9
Adana                               113
Addis Ababa                          43
Adelaide                             38
Agra                                 54
Aguascalientes                        6
Ahmedabad                          1225
Akron                               136
Alameda                               8
Alba                                  7
Alexandria                          156
Alicante                             27
Almaty                               21
Almelo                               11
Almeria                              53
Almetyevsk                           36
Amberg                               37
Amersfoort                            7
Amherst                            

In [60]:
df2.groupby("city").size()

city
(not set)                         34788
Aalborg                              32
Aalen                                 9
Abu Dhabi                            48
Adana                               152
Addis Ababa                          61
Adelaide                              6
Agra                                 27
Ahmedabad                          1256
Akron                               119
Alameda                               6
Alba Iulia                           21
Alexandria                          337
Alicante                             17
Allendale Charter Township            8
Almaty                               12
Almeria                             105
Alton                                 6
Altrincham                           14
Amberg                               12
Amiens                                6
Amsterdam                          1682
Amã                                  64
Anaheim                               4
Anchorage                          

In [61]:
df.groupby("cityId").size()

cityId
not available in demo dataset    803178
dtype: int64

In [62]:
df2.groupby("cityId").size()

cityId
not available in demo dataset    905160
dtype: int64

In [63]:
df.groupby("networkDomain").size()

networkDomain
(not set)                  233532
0101.net.au                     2
012.net.il                    224
018.net.il                     87
02.net                        458
099.net.il                     21
0pb.org                         1
0rbitel.net                     7
0und1.net                       1
1-pipexbd.com                   2
10035.com.cn                    1
100megabit.ru                  12
100tb.com                      21
101netlink.com                  1
103-ebonenet.com                1
10call.com                      1
10gbps.io                      39
10net.net                       2
10outof10winning.net            1
111-tataidc.co.in             208
115-tataidc.co.in               8
1151518.com                     2
122-airtelbroadband.in        154
123.net                        21
123ce.fr                        4
123multimedia.com               1
123planosdesaude.com.br         7
130.2                           1
133513.com                      1


In [64]:
df2.groupby("networkDomain").size()

networkDomain
(not set)                      265517
0101.net.au                         2
012.net.il                        231
018.net.il                         89
01net.com                           2
02.net                            589
04.1-512mb-nyc1-01                  1
04.4-x64                            1
099.net.il                         35
0beta                               1
0rbitel.net                         5
1-bk.ru                             1
1-pipexbd.com                       1
10-42.net                           1
10035.com.cn                        3
100megabit.ru                      14
100porcento.net                     1
100tb.com                          37
101email.com                        2
101netlink.com                      2
103-static-fiberlink.net.pk         1
109.235                             1
10gbps.io                          67
10net.net                           1
11.com                              1
111-tataidc.co.in                 21

In [65]:
df.groupby("latitude").size()

latitude
not available in demo dataset    803178
dtype: int64

In [66]:
df2.groupby("latitude").size()

latitude
not available in demo dataset    905160
dtype: int64

In [68]:
df.groupby("longitude").size()

longitude
not available in demo dataset    803178
dtype: int64

In [69]:
df2.groupby("longitude").size()

longitude
not available in demo dataset    905160
dtype: int64

In [70]:
df.groupby("networkLocation").size()

networkLocation
not available in demo dataset    803178
dtype: int64

In [71]:
df2.groupby("networkLocation").size()

networkLocation
not available in demo dataset    905160
dtype: int64

In [72]:
df.groupby("visits").size()

visits
1    803178
dtype: int64

In [73]:
df2.groupby("visits").size()

visits
1    905160
dtype: int64

In [74]:
df.groupby("hits").size()

hits
1      406121
10       9432
100        24
101        20
102        23
103        16
104        28
105        23
106        19
107        11
108        17
109        18
11       8093
110        16
111        15
112        10
113        18
114        12
115        16
116        18
117         9
118         6
119        12
12       6903
120        11
121        11
122        14
123         7
124         8
125        10
        ...  
72         73
73         93
74         81
75         69
76         77
77         58
78         53
79         46
8       13863
80         49
81         54
82         56
83         49
84         54
85         38
86         45
87         26
88         34
89         38
9       11501
90         35
91         37
92         23
93         31
94         34
95         31
96         33
97         16
98         20
99         24
Length: 243, dtype: int64

In [75]:
df2.groupby("hits").size()

hits
1      457943
10      10874
100        36
101        30
102        23
103        26
104        19
105        22
106        24
107        18
108        19
109        19
11       9413
110        23
111        17
112        25
113         8
114        16
115        14
116        17
117        15
118        17
119        11
12       7974
120        17
121        12
122        17
123        15
124        12
125        12
        ...  
72         90
73         87
74         87
75         76
76         74
77         65
78         66
79         77
8       15854
80         70
81         68
82         53
83         52
84         63
85         47
86         47
87         50
88         35
89         39
9       13498
90         36
91         34
92         33
93         32
94         45
95         35
96         25
97         28
98         33
99         18
Length: 261, dtype: int64

In [76]:
df.groupby("pageviews").size()

pageviews
1      411672
10       8925
100         7
101         6
102         6
103         3
104         5
105         3
106         6
107         4
108         7
11       7649
110         4
111         5
112         6
113         6
114         3
115         8
116         3
117         3
118         4
119         3
12       6238
120         4
121         4
122         1
123         3
124         2
125         4
126         4
        ...  
72         30
73         29
74         27
75         29
76         21
77         21
78         23
79         13
8       13809
80         18
81         18
82         15
83          9
84         12
85         13
86         14
87          9
88         10
89         13
9       11254
90         11
91         11
92         13
93          6
94          6
95          5
96          9
97          8
98          9
99          7
Length: 185, dtype: int64

In [77]:
df2.groupby("pageviews").size()

pageviews
1      464656
10      10303
100        10
101         7
102         8
103         8
104         8
105         7
106         6
107         7
108         6
109         7
11       8861
110         9
111         5
112         5
113         7
114         5
115         2
116         4
117         4
118         7
119         2
12       7176
120         2
121         6
122         2
123         3
124         4
125         1
        ...  
72         25
73         35
74         26
75         33
76         29
77         23
78         23
79         21
8       15777
80         28
81         18
82         14
83         19
84         12
85         17
86         11
87         12
88         16
89         18
9       12899
90         19
91         10
92          9
93         18
94         12
95         11
96         11
97          5
98          9
99          7
Length: 195, dtype: int64

In [78]:
df.groupby("bounces").size()

bounces
1    409613
dtype: int64

In [79]:
df2.groupby("bounces").size()

bounces
1    461965
dtype: int64

In [80]:
df.groupby("newVisits").size()

newVisits
1    616774
dtype: int64

In [81]:
df2.groupby("newVisits").size()

newVisits
1    690656
dtype: int64

In [82]:
df.groupby("sessionQualityDim").size()

sessionQualityDim
1     313628
10      1004
11       887
12       762
13       675
14       597
15       561
16       522
17       437
18       423
19       434
2      22792
20       385
21       386
22       397
23       408
24       326
25       357
26       344
27       317
28       298
29       329
3       7388
30       294
31       319
32       255
33       246
34       254
35       257
36       278
       ...  
72       226
73       185
74       213
75       188
76       218
77       199
78       214
79       207
8       1454
80       202
81       183
82       162
83       172
84       173
85       157
86       146
87       137
88       133
89        80
9       1194
90        71
91        67
92        62
93        44
94        54
95        39
96        18
97        10
98         3
99         1
Length: 99, dtype: int64

In [83]:
df2.groupby("sessionQualityDim").size()

sessionQualityDim
1      403933
10       1378
100         2
11       1183
12       1041
13        914
14        815
15        751
16        699
17        626
18        572
19        590
2       29725
20        565
21        574
22        547
23        471
24        443
25        459
26        507
27        449
28        456
29        384
3        9678
30        402
31        414
32        367
33        379
34        332
35        363
        ...  
72        293
73        296
74        272
75        297
76        268
77        273
78        282
79        258
8        1883
80        287
81        268
82        254
83        273
84        226
85        210
86        181
87        161
88        125
89        112
9        1535
90        111
91         93
92         65
93         58
94         58
95         55
96         34
97         17
98          6
99          4
Length: 100, dtype: int64

In [84]:
df.groupby("timeOnSite").size()

timeOnSite
1         714
10       3862
100      1144
1000       41
1001       28
1002       24
1003       38
1004       40
1005       47
1006       33
10068       1
1007       30
1008       40
1009       43
101      1142
1010       46
1011       29
1012       21
1013       39
1014       45
1015       38
1016       32
1017       40
1018       29
1019       27
102      1175
1020       41
1021       27
1022       37
1023       36
         ... 
975        26
976        38
977        37
9772        1
978        29
979        28
98       1056
980        31
981        28
982        35
983        28
9837        1
984        33
985        39
986        40
987        34
988        30
989        39
99       1051
990        46
991        39
992        39
993        30
994        45
995        36
996        44
9963        1
997        57
998        33
999        30
Length: 3993, dtype: int64

In [85]:
df2.groupby("timeOnSite").size()

timeOnSite
1         772
10       4255
100      1307
1000       36
1001       41
1002       32
1003       41
1004       44
10046       1
1005       39
1006       48
1007       44
10076       1
1008       42
1009       35
101      1263
1010       46
1011       45
1012       36
1013       48
1014       45
1015       42
1016       35
1017       35
1018       31
1019       34
102      1273
1020       44
1021       38
1022       37
         ... 
975        38
976        44
977        33
978        37
979        49
98       1188
980        39
981        44
982        33
9826        1
983        29
984        50
985        40
9858        1
986        46
987        41
988        34
989        49
99       1222
990        32
991        34
992        64
993        47
9932        1
994        31
995        47
996        35
997        53
998        50
999        39
Length: 4203, dtype: int64

In [86]:
df.groupby("transactions").size()

transactions
1    8479
2     202
3      26
4      11
5       5
6       3
7       2
8       2
dtype: int64

In [87]:
df2.groupby("transactions").size()

transactions
1     9569
10       1
12       2
15       1
2      218
21       1
25       1
3       17
4        6
5        6
6        3
7        3
8        1
dtype: int64

In [88]:
df.groupby("transactionRevenue").size()

transactionRevenue
10000000       3
100000000     30
100050000      1
1000780000     1
100220000      1
100440000      1
1004490000     1
100450000      1
100700000      1
100720000      1
100730000      1
100760000      1
100770000      9
100780000      1
1007830000     1
100790000      1
100900000      1
100920000      1
100940000      1
101090000      1
101230000      1
101490000      1
101570000      1
101630000      1
101940000      2
1019400000     1
101950000      2
1019500000     1
101970000      2
10200000       1
              ..
98900000       1
98990000       6
990000         1
9900000        2
99050000       1
99130000       1
99140000       1
99150000       1
99170000       1
99260000       1
99290000       1
99400000       1
99490000       1
9950000        2
99500000       2
99530000       2
99540000       1
995450000      1
9960000        2
99600000       1
996430000      1
9980000        8
99800000       2
99850000       1
9990000        9
99900000       1
99950000    

In [89]:
df2.groupby("transactionRevenue").size()

transactionRevenue
10000          1
1000000        2
10000000       6
100000000     19
1000290000     1
1001590000     1
100260000      1
100350000      1
100440000      2
100560000      1
100570000      1
10070000       1
100740000      1
100760000      3
100770000      5
100780000      1
100800000      1
1008480000     1
100930000      2
100950000      1
100960000      1
101000000      1
1012190000     1
101230000      1
101300000      2
101440000      1
101460000      1
101480000      1
101500000      1
101540000      2
              ..
99000000       1
99130000       2
99160000       2
99170000       1
9920000        1
99250000       2
9925110000     1
992800000      1
993390000      1
994650000      1
9950000        2
99500000       1
995000000      1
995680000      2
9960000        3
9970000        4
99710000       1
99720000       1
99750000       2
99760000       1
9980000        6
99850000       1
998750000      1
9990000       18
99950000       5
99960000       3
99970000    

In [101]:
sum([v for _,v in df.groupby("transactionRevenue").size().iteritems()]) + sum([v for _,v in df2.groupby("transactionRevenue").size().iteritems()])

18514

In [90]:
df.groupby("totalTransactionRevenue").size()

totalTransactionRevenue
100000000      1
100130000      1
100140000      1
100170000      1
1003430000     1
100370000      1
100450000      1
100520000      1
100540000      1
100550000      1
1005500000     1
100630000      1
100670000      1
1006780000     1
100700000      1
100760000      1
100770000      1
100780000      1
100810000      1
100840000      1
1008490000     1
100880000      1
100910000      1
100940000      2
100950000      1
100960000      1
100970000      1
100980000      2
100990000      2
10100000       1
              ..
985730000      1
987300000      1
98870000       1
98930000       2
98940000       2
98950000       1
98960000       1
98970000       2
98980000       5
98990000       1
99100000       1
99160000       1
991850000      1
99240000       1
99410000       1
99550000       1
99600000       1
99690000       1
9970000        1
99740000       1
9980000        3
99820000       1
99830000       1
998450000      1
9990000       12
99950000       1
9996000

In [96]:
df2.groupby("totalTransactionRevenue").size()

totalTransactionRevenue
10000000      3
100010000     1
100080000     1
1001000000    1
1001700000    1
1001750000    1
1001900000    1
100210000     1
1002290000    1
100250000     1
1002590000    1
100260000     1
100370000     1
100380000     1
100400000     1
100470000     1
100620000     1
100640000     1
100690000     1
100760000     1
100800000     2
100950000     2
100960000     1
100970000     1
100990000     6
101000000     2
101110000     1
101160000     1
101260000     1
101270000     1
             ..
98940000      1
98950000      1
98960000      1
98970000      5
98980000      2
98990000      1
99040000      1
99070000      1
99130000      1
99150000      1
99210000      2
99340000      1
994390000     1
99450000      1
99570000      1
9960000       2
996120000     1
996650000     1
996680000     1
9970000       3
99760000      1
99770000      1
9980000       3
99800000      2
99880000      1
998800000     1
9990000       9
99940000      1
999680000     1
99970000      2


In [100]:
sum([v for _,v in df.groupby("totalTransactionRevenue").size().iteritems()]) + sum([v for _,v in df2.groupby("totalTransactionRevenue").size().iteritems()])

18514

In [102]:
df.groupby("campaign").size()

campaign
"google + redesign/Accessories March 17" 1 Day                               3
"google + redesign/Accessories March 17" All Users                         241
"google + redesign/Accessories March 17" All Users Similar Audiences       514
(direct)                                                                     6
(not set)                                                               756278
1000557 | GA | US | en | Hybrid | GDN Remarketing                         6513
1000557 | GA | US | en | Hybrid | GDN Text+Banner | AS                   11261
AW - Accessories                                                          3504
AW - Apparel                                                                25
AW - Dynamic Search Ads Whole Site                                        6842
AW - Electronics                                                            28
All Products                                                                 1
Data Share Promo                           

In [103]:
df2.groupby("campaign").size()

campaign
"google + redesign/Accessories March 17" 1 Day                               2
"google + redesign/Accessories March 17" All Users                         321
"google + redesign/Accessories March 17" All Users Similar Audiences       665
(direct)                                                                     6
(not set)                                                               848249
1000557 | GA | US | en | Hybrid | GDN Remarketing                         8636
1000557 | GA | US | en | Hybrid | GDN Text+Banner | AS                   13149
AW - Accessories                                                          4468
AW - Apparel                                                                19
AW - Dynamic Search Ads Whole Site                                        8304
AW - Electronics                                                            47
All Products                                                                 3
Data Share                                 

In [104]:
df.groupby("source").size()

source
(direct)                           280649
(not set)                              29
Partners                            14900
admin.globalaccess.com                  2
adwords.google.com                     75
allo.google.com                         3
amazon.com                              8
amp.reddit.com                          3
analytics.google.com                16996
aol                                     6
apis.mail.yahoo.com                     4
apps.rackspace.com                      1
ar.search.yahoo.com                     1
arstechnica.com                        14
ask                                   188
au.search.yahoo.com                    14
aulagrado.esic.edu                     17
ausdroid.net                            1
away.vk.com                           240
baidu                                2341
baidu.com                               6
basecamp.com                            2
bing                                 1536
bing.com                   

In [105]:
df2.groupby("source").size()

source
(direct)                           285326
(not set)                              41
9to5google.com                          2
Partners                            18031
ad.doubleclick.net                      3
adwords.google.com                    102
amazon.com                              9
amp.reddit.com                         10
analytics.google.com                20440
aol                                     7
apis.mail.yahoo.com                     4
ar.search.yahoo.com                     2
arstechnica.com                        22
ask                                   229
au.search.yahoo.com                    10
aulagrado.esic.edu                     28
ausdroid.net                            1
away.vk.com                           373
baidu                                2630
baidu.com                               4
basecamp.com                            1
bbhosted.cuny.edu                       8
bing                                 1832
bing.com                   

In [106]:
df.groupby("medium").size()

medium
(none)       280640
(not set)        50
affiliate     14892
cpc           34023
cpm            4308
organic      258578
referral     210687
dtype: int64

In [107]:
df2.groupby("medium").size()

medium
(none)       285317
(not set)        84
affiliate     18023
cpc           41580
cpm            4674
organic      333206
referral     222276
dtype: int64

In [108]:
df.groupby("keyword").size()

keyword
"waterproof backpack"                               1
"youtube"                                           1
(Remarketing/Content targeting)                  2718
(User vertical targeting)                       11912
(automatic matching)                             8094
(content targeting)                                19
(not provided)                                 248389
+Android +Merchandise                               1
+Chrome +Products                                   1
+Google +Gear                                       8
+Google +Merchandise                              188
+Google +Products                                  77
+Google +Swag                                      67
+Google +accessories                               15
+Google +glassware                                  2
+Google +pet                                        2
+Google cap                                         1
+YouTube +Gear                                      1
+YouTube +Merchandis

In [109]:
df2.groupby("keyword").size()

keyword
(/google+redesign/shop+by+brand/youtube                       1
(Remarketing/Content targeting)                            3108
(User vertical targeting)                                 14006
(automatic matching)                                      10370
(content targeting)                                          23
(not provided)                                           320545
*board/shop lifestyle/cap                                     1
+Android +Gear                                                2
+Chrome +Merchandise                                          2
+Google +Gear                                                13
+Google +Merchandise                                        270
+Google +Products                                           115
+Google +Swag                                                62
+Google +accessories                                         13
+Google +drinkware                                            2
+Google +glassware              

In [ ]:
df.groupby("adwordsClickInfo").size()

In [ ]:
df2.groupby("adwordsClickInfo").size()

In [111]:
df.groupby("referralPath").size()

referralPath
/                                                                                            63896
//r/golang/comments/4zk0qy/in_serious_need_of_some_gopher_golang_swag/                           1
//yt/about/fr/                                                                                   1
/0FRjgIcmaR                                                                                      1
/0xpEZvuoUz                                                                                      4
/2015/03/11/google-merch-store-new-url/                                                        155
/2015/06/08/the-googleplex-really-is-one-of-the-most-fun-places-to-work/                         1
/2017/06/360.html                                                                                1
/2jOketZr1r                                                                                      1
/3oIBwdLr5K                                                                                     

In [112]:
df2.groupby("referralPath").size()

referralPath
/                                                                                            74397
//QwaujgsLA4                                                                                     1
/02BvQNNV8S                                                                                      1
/0x50/items/208dcb46005533a9d889                                                                 5
/0xpEZvuoUz                                                                                      6
/19JPpPMW0p                                                                                      6
/1uXVFaurAg                                                                                      1
/2015/03/11/google-merch-store-new-url/                                                        158
/2015/03/11/google-merch-store-new-url/amp/                                                      2
/2015/06/08/the-googleplex-really-is-one-of-the-most-fun-places-to-work/                        

In [113]:
df.groupby("isTrueDirect").size()

isTrueDirect
True    251026
dtype: int64

In [114]:
df2.groupby("isTrueDirect").size()

isTrueDirect
True    283493
dtype: int64

In [115]:
df.groupby("adContent").size()

adContent
20% discount                             38
Accessories Placement 336 x 280           8
Ad from 11/3/16                         261
Ad from 11/7/16                          45
Ad from 12/13/16                        296
Ad from 12/15/17                        326
Ad from 12/21/16                          9
BQ                                       68
Bags 160 x 600                            8
Bags 300x250                             40
Bags 336x280                             28
Display Ad created 11/17/14              40
Display Ad created 3/11/14             1060
Display Ad created 3/11/15              232
Drinkware 120x600                         2
Drinkware 336x280                        43
First Full Auto Template Test Ad         53
Full auto ad IMAGE ONLY                 410
Full auto ad NATIVE ONLY                  1
Full auto ad TEXT ONLY                   10
Full auto ad TEXT/NATIVE                  2
Full auto ad with Primary Color           2
GA Help Center        

In [116]:
df2.groupby("adContent").size()

adContent
20% discount                             37
Accessories Placement 336 x 280           5
Ad from 11/3/16                         228
Ad from 11/7/16                          78
Ad from 12/13/16                        314
Ad from 12/15/17                        527
Ad from 12/21/16                         39
Ad from 2/17/17                           3
BQ                                      219
Bags 160 x 600                            5
Bags 300x250                             50
Bags 336x280                             21
Display Ad created 11/17/14              10
Display Ad created 3/11/14             1284
Display Ad created 3/11/15              160
Drinkware 120x600                         4
Drinkware 336x280                        53
First Full Auto Template Test Ad         34
Free Shipping!                            2
Full auto ad IMAGE ONLY                 412
Full auto ad NATIVE ONLY                  2
Full auto ad TEXT ONLY                    6
Full auto ad TEXT/NATI

In [127]:
keep = ["channelGrouping", "date", "customDimensions", "visitNumber",
        "visitStartTime","browser", "operatingSystem", "isMobile", "deviceCategory",
        "continent", "subContinent", "country", "region", "city", "metro", "networkDomain",
        "hits", "pageviews","sessionQualityDim", "timeOnSite", "transactions", "transactionRevenue",
        "totalTransactionRevenue",'campaign','source', 'medium','keyword',  'referralPath','isTrueDirect','adContent']

In [131]:
to_drop = ["socialEngagementType", "fullVisitorId", "visitId", "browserVersion", "browserSize","mobileDeviceBranding", "mobileDeviceModel","mobileInputSelector", "mobileDeviceInfo",'flashVersion',
 'language','screenColors','screenResolution',"cityId", "mobileDeviceMarketingName", "operatingSystemVersion",
"latitude", "longitude", "networkLocation", "visits", "bounces", "newVisits", "adwordsClickInfo"]

In [132]:
len(keep) + len(to_drop)

53

In [133]:
len(column_names)

53

In [134]:
df.shape, df2.shape

((803178, 53), (905160, 53))

In [135]:
df.drop(to_drop,axis=1, inplace=True)

In [136]:
df2.drop(to_drop,axis=1, inplace=True)

In [137]:
df.shape, df2.shape

((803178, 30), (905160, 30))

In [ ]:
#starting to process hits


# DONT RUN NEXT 6 CELLS

In [138]:
one_row = pd.read_csv("train_v2.csv", nrows=1)
one_row = list(one_row)
one_row

In [142]:
%%time 
no_of_rows = 803178
# converter applies a converter function while you are loading the data, you are loading it as
# json.loadstring = json.loads() for each of those columns in the list
train_set = pd.read_csv("train_v2.csv", usecols=[i for i in one_row if i== 'hits'], nrows = no_of_rows)

CPU times: user 1min 13s, sys: 16.2 s, total: 1min 29s
Wall time: 1min 34s


In [152]:
y = eval(train_set["hits"][0])
y


[{'hitNumber': '1',
  'time': '0',
  'hour': '17',
  'minute': '0',
  'isInteraction': True,
  'isEntrance': True,
  'isExit': True,
  'referer': 'https://www.google.co.uk/search?q=water+bottle&ie=utf-8&num=100&oe=utf-8&hl=en&gl=GB&uule=w+CAIQIFISCamRx0IRO1oCEXoliDJDoPjE&glp=1&gws_rd=cr&fg=1',
  'page': {'pagePath': '/google+redesign/bags/water+bottles+and+tumblers',
   'hostname': 'shop.googlemerchandisestore.com',
   'pageTitle': 'Water Bottles & Tumblers | Drinkware | Google Merchandise Store',
   'pagePathLevel1': '/google+redesign/',
   'pagePathLevel2': '/bags/',
   'pagePathLevel3': '/water+bottles+and+tumblers',
   'pagePathLevel4': ''},
  'transaction': {'currencyCode': 'USD'},
  'item': {'currencyCode': 'USD'},
  'appInfo': {'screenName': 'shop.googlemerchandisestore.com/google+redesign/bags/water+bottles+and+tumblers',
   'landingScreenName': 'shop.googlemerchandisestore.com/google+redesign/bags/water+bottles+and+tumblers',
   'exitScreenName': 'shop.googlemerchandisestore.c

In [ ]:
for i, j in train_set["hits"].iteritems():
    train_set["hits"][i] = eval(j)

In [ ]:
for i, j in train_set["hits"].iteritems():
    train_set["hits"][i] = train_set["hits"][i][0]
    train_set["hits"][i] = json.dumps(train_set["hits"][i])

In [ ]:
train_set = train_set.join(json.dumps()).apply(json.loads).apply(pd.Series))

In [167]:
df.head()

,channelGrouping,customDimensions,date,visitNumber,visitStartTime,browser,operatingSystem,isMobile,deviceCategory,continent,...,transactions,transactionRevenue,totalTransactionRevenue,campaign,source,medium,keyword,referralPath,isTrueDirect,adContent
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,1,1508198450,Firefox,Windows,False,desktop,Europe,...,NaN,NaN,NaN,(not set),google,organic,water bottle,NaN,NaN,NaN
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,6,1508176307,Chrome,Chrome OS,False,desktop,Americas,...,NaN,NaN,NaN,(not set),sites.google.com,referral,NaN,/a/google.com/transportation/mtv-services/bike...,NaN,NaN
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,1,1508201613,Chrome,Android,True,mobile,Americas,...,NaN,NaN,NaN,(not set),(direct),(none),NaN,NaN,True,NaN
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,1,1508169851,Chrome,Windows,False,desktop,Asia,...,NaN,NaN,NaN,(not set),google,organic,(not provided),NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,1,1508190552,Chrome,Windows,False,desktop,Americas,...,NaN,NaN,NaN,(not set),google,organic,(not provided),NaN,NaN,NaN


In [168]:
df2.head()

,channelGrouping,customDimensions,date,visitNumber,visitStartTime,browser,operatingSystem,isMobile,deviceCategory,continent,...,transactions,transactionRevenue,totalTransactionRevenue,campaign,source,medium,keyword,isTrueDirect,referralPath,adContent
0,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,2,1507250518,Chrome,Android,True,mobile,Americas,...,NaN,NaN,NaN,(not set),google,organic,(not provided),True,NaN,NaN
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,1,1507250122,Safari,iOS,True,tablet,Americas,...,NaN,NaN,NaN,(not set),google,organic,(not provided),NaN,NaN,NaN
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,1,1507237813,Chrome,Windows,False,desktop,Americas,...,NaN,NaN,NaN,(not set),google,organic,(not provided),NaN,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,1,1507250491,Chrome,Android,True,mobile,Americas,...,NaN,NaN,NaN,(not set),google,organic,(not provided),NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,1,1507243409,Safari,iOS,True,mobile,Americas,...,NaN,NaN,NaN,(not set),google,organic,(not provided),NaN,NaN,NaN


In [169]:
tf = pd.read_pickle("preprocessed_test_set_without_hits.pkl")

In [170]:
tf.drop(to_drop, axis=1, inplace=True)

In [171]:
tf.shape

(401589, 30)

In [172]:
# convert the int64 datatype of date to meaningful dates of typedatetime64
df.date = pd.to_datetime(df.date,format='%Y%m%d')

In [174]:
# convert the int64 datatype of date to meaningful dates of typedatetime64
df2.date = pd.to_datetime(df2.date,format='%Y%m%d')

In [175]:
# convert the int64 datatype of date to meaningful dates of typedatetime64
tf.date = pd.to_datetime(tf.date,format='%Y%m%d')

In [177]:
df["is_month_start"] = pd.DataFrame(df.date.dt.is_month_start)
df["is_month_end"] = pd.DataFrame(df.date.dt.is_month_end)
df["is_year_end"] = pd.DataFrame(df.date.dt.is_year_end)
df["is_year_start"] = pd.DataFrame(df.date.dt.is_year_start)
df["is_quarter_end"] = pd.DataFrame(df.date.dt.is_quarter_end)
df["is_quarter_start"] = pd.DataFrame(df.date.dt.is_quarter_start)
df["weekofyear"] = pd.DataFrame(df.date.dt.weekofyear)

In [179]:
df.shape

(803178, 37)

In [180]:
df2.shape, tf.shape

((905160, 30), (401589, 30))

In [181]:
df2["is_month_start"] = pd.DataFrame(df2.date.dt.is_month_start)
df2["is_month_end"] = pd.DataFrame(df2.date.dt.is_month_end)
df2["is_year_end"] = pd.DataFrame(df2.date.dt.is_year_end)
df2["is_year_start"] = pd.DataFrame(df2.date.dt.is_year_start)
df2["is_quarter_end"] = pd.DataFrame(df2.date.dt.is_quarter_end)
df2["is_quarter_start"] = pd.DataFrame(df2.date.dt.is_quarter_start)
df2["weekofyear"] = pd.DataFrame(df2.date.dt.weekofyear)

In [182]:
tf["is_month_start"] = pd.DataFrame(tf.date.dt.is_month_start)
tf["is_month_end"] = pd.DataFrame(tf.date.dt.is_month_end)
tf["is_year_end"] = pd.DataFrame(tf.date.dt.is_year_end)
tf["is_year_start"] = pd.DataFrame(tf.date.dt.is_year_start)
tf["is_quarter_end"] = pd.DataFrame(tf.date.dt.is_quarter_end)
tf["is_quarter_start"] = pd.DataFrame(tf.date.dt.is_quarter_start)
tf["weekofyear"] = pd.DataFrame(tf.date.dt.weekofyear)

In [183]:
df2.shape, tf.shape

((905160, 37), (401589, 37))

In [247]:
df.to_pickle("train_set.pkl")

In [248]:
df2.to_pickle("train_set2.pkl")

In [249]:
tf.to_pickle("test_set.pkl")

In [189]:
df["visitStartTime"] = pd.DataFrame(pd.to_datetime(df["visitStartTime"], unit="s"))

In [190]:
df2["visitStartTime"] = pd.DataFrame(pd.to_datetime(df2["visitStartTime"], unit="s"))

In [215]:
tf["visitStartTime"] = pd.DataFrame(pd.to_datetime(tf["visitStartTime"], unit="s"))

In [192]:
df.drop(["timestamp"], axis=1, inplace=True)

In [193]:
df.shape, df2.shape, tf.shape

((803178, 37), (905160, 37), (401589, 37))

In [197]:
df.head()

,channelGrouping,customDimensions,date,visitNumber,visitStartTime,browser,operatingSystem,isMobile,deviceCategory,continent,...,referralPath,isTrueDirect,adContent,is_month_start,is_month_end,is_year_end,is_year_start,is_quarter_end,is_quarter_start,weekofyear
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",2017-10-16,1,2017-10-17 00:00:50,Firefox,Windows,False,desktop,Europe,...,NaN,NaN,NaN,False,False,False,False,False,False,42
1,Referral,"[{'index': '4', 'value': 'North America'}]",2017-10-16,6,2017-10-16 17:51:47,Chrome,Chrome OS,False,desktop,Americas,...,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,False,False,False,False,False,False,42
2,Direct,"[{'index': '4', 'value': 'North America'}]",2017-10-16,1,2017-10-17 00:53:33,Chrome,Android,True,mobile,Americas,...,NaN,True,NaN,False,False,False,False,False,False,42
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",2017-10-16,1,2017-10-16 16:04:11,Chrome,Windows,False,desktop,Asia,...,NaN,NaN,NaN,False,False,False,False,False,False,42
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",2017-10-16,1,2017-10-16 21:49:12,Chrome,Windows,False,desktop,Americas,...,NaN,NaN,NaN,False,False,False,False,False,False,42


In [211]:
df["hourofday"] = df.visitStartTime.dt.hour


In [ ]:
df2["hourofday"] = df2.visitStartTime.dt.hour


In [217]:
tf["hourofday"] = tf.visitStartTime.dt.hour

In [222]:
df.drop(["date", "visitStartTime"], axis=1, inplace=True)
df2.drop(["date", "visitStartTime"], axis=1, inplace=True)
tf.drop(["date", "visitStartTime"], axis=1, inplace=True)

In [250]:
df.shape, df2.shape, tf.shape

((803178, 35), (905160, 35), (401589, 35))

In [232]:
df.head()

,channelGrouping,customDimensions,visitNumber,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,...,referralPath,adContent,is_month_start,is_month_end,is_year_end,is_year_start,is_quarter_end,is_quarter_start,weekofyear,hourofday
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",1,Firefox,Windows,False,desktop,Europe,Western Europe,Germany,...,NaN,NaN,False,False,False,False,False,False,42,0
1,Referral,"[{'index': '4', 'value': 'North America'}]",6,Chrome,Chrome OS,False,desktop,Americas,Northern America,United States,...,/a/google.com/transportation/mtv-services/bike...,NaN,False,False,False,False,False,False,42,17
2,Direct,"[{'index': '4', 'value': 'North America'}]",1,Chrome,Android,True,mobile,Americas,Northern America,United States,...,NaN,NaN,False,False,False,False,False,False,42,0
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",1,Chrome,Windows,False,desktop,Asia,Western Asia,Turkey,...,NaN,NaN,False,False,False,False,False,False,42,16
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",1,Chrome,Windows,False,desktop,Americas,Central America,Mexico,...,NaN,NaN,False,False,False,False,False,False,42,21


In [228]:
df.drop(["isTrueDirect"], axis=1, inplace=True)
df2.drop(["isTrueDirect"], axis=1, inplace=True)
tf.drop(["isTrueDirect"], axis=1, inplace=True)

In [ ]:
#NOT REQUIRED
counter = 0
for column in df.keys():
    total = []
    xaxis = []
    a = df.groupby(column).size()
    b = df2.groupby(column).size()
    for aa, bb in zip(a.iteritems(), b.iteritems()):
        total.append(aa[1] + bb[1])
        xaxis.append(aa[0])
        
    y_pos = np.arange(len(xaxis))    
        
    plt.barh(y_pos, total, align = "center", alpha= 0.5)
    plt.yticks(y_pos, xaxis)
    plt.title(column_names[counter] + " Spread")
    plt.show()
    counter+=1

In [ ]:
column_names

In [238]:
df["transactionRevenue"].fillna(0, inplace=True)
df2["transactionRevenue"].fillna(0, inplace=True)



In [239]:
print("Variables not in test but in train : ", set(df.columns).difference(set(tf.columns)))

Variables not in test but in train :  set()


In [245]:
sum([v for _, v in tf.groupby("transactionRevenue").size().iteritems()])

4594

In [262]:
df[df.transactionRevenue == 0]
df.transactionRevenue.values

array([0, 0, 0, ..., 0, 0, 0], dtype=object)